In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import csv

driver = webdriver.Chrome('chromedriver')
time.sleep(1)
login_Process();
time.sleep(1)
loops = '1'
while(loops=='1'):
    loops = 0
    id_Num = input('輸入三碼ID:')
    time.sleep(1)
    select_Id(id_Num);
    print('{} 完成 , 若需檢查其他項目請輸入1,結束請關閉。'.format(id_Num))
    loops = (str)(input('請輸入 : '))
drive.close()

def login_Process():
    print('opening chrome..')
    driver.get('http://192.168.3.64/login') 
    print('typing account & pw')
    user_name = 'dnakesehr' #input('帳號:')
    password = 'emotibot' #input('密碼:')

    elem = driver.find_element_by_name("name")
    elem.clear()
    elem.send_keys(user_name)

    elem = driver.find_element_by_name("password")
    elem.clear()
    elem.send_keys(password)
    driver.find_element_by_xpath("//button[@type='submit']").click()
    print('successfully logined to account : ' + user_name + '!')

def select_Id(id_Num):
    print('now start to do num : '+ (str)(id_Num))
    url = 'http://192.168.3.64/tasks/{}/hits/list?page={}'
    driver.get(url.format(id_Num,1))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    counts = 0
    lists = ['id','href','condition','worder','workTime','editedTime','Reviewer','recordsNum']
    k = soup.select('ul[class="pagination"]')
    for i in k[0].select('a'):
        #print(i.get('href'))
        counts=counts+1
    print('總頁數 : '+(str)(counts))    #總頁數
    list_url = []
    with open('C:/Users/{}_data.csv'.format(id_Num), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for pages in range(counts):
            driver.get(url.format(id_Num,pages+1))
            soup = BeautifulSoup(driver.page_source, 'lxml')
            count = -1
            print('第 '+ (str)(pages+1)+' 頁')
            lists = ['id','href','condition','worder','workTime','editedTime','Reviewer','recordsNum']
            for i in soup.select('tr'):
                #href = i.select_one('td') #抓ID  
                types = 1   #全類別共八個資訊 {id,href,condition,worder,workTime,editedTime,Reset,Reviewer}
                #print(pages+1)
                for j in i.select('td'):
                    if(types%8==1):  #id
                        lists[0] = j.text
                        #print('id = '+lists[0])
                    elif(types%8==2 and len(j.text)<100):  #href
                        #print(j.text+(str)(len(j.text)))
                        lists[1] = j.select_one('a').get('href')
                        list_url.append(lists[1])
                        #print('網址 = '+lists[1]) #網址
                    elif(types%8==2 and len(j.text)>=100):
                        lists[1] = ''
                        #print(j.text+(str)(len(j.text)))
                    elif(types%8==3):  #Condition
                        lists[2] = j.text
                        #print('Condition = '+lists[2])
                    elif(types%8==4):  #Worker
                        lists[3] = j.text
                        #print('Worker = '+lists[3])
                    elif(types%8==5):  #工作時間
                        k = j.text.split(" ")
                        lists[4] = k[0]
                        #print('工作時間 = '+lists[4] +' mins')
                    elif(types%8==6):  #最後修改時間
                        lists[5] = j.text
                        #print('最後修改時間 = '+lists[5])
                    elif(types%8==0):  #Reviewed By
                        lists[6] = j.text.strip()
                        #print('Reviewed By = '+lists[6])
                    lists[7] = 'none'
                    types=types+1
                count=count+1
                writer.writerow(lists)
            print ('此頁總數 : ' +(str)(count))
    print('starting counting records ..')
    recordsNum(list_url,id_Num)
    
def recordsNum(list_url,id_Num):
    num=0
    with open('C:/Users/{}_data.csv'.format(id_Num), 'r+', newline='') as csvfile,open('C:/Users/{}_dataOut.csv'.format(id_Num), 'w', newline='') as out:
        writer = csv.writer(out)
        csv_read = list(csv.reader(csvfile))
        writer.writerow(csv_read[0])
        for line in csv_read:
            if(line[7]=='none'):
                if(line[1]!=''): 
                    driver.get(list_url[num])
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    time.sleep(0.3)
                    while(True):
                        if(soup.select('#length')[0].text!=''):
                            break
                        time.sleep(0.2)
                        soup = BeautifulSoup(driver.page_source, 'lxml')
                    a=0
                    p = soup.select_one('table[class="table table-hover table-fixedheader"]').select_one('tbody').select('tr')
                    for i in p:
                        a=a+1
                    line[7]=a-1
                    num=num+1
                    print('now writing number:'+(str)(num)+' which is '+(str)(line[7]))
                    time.sleep(0.1)
                writer.writerow(line)

NameError: name 'login_Process' is not defined